In [ ]:
class InMemoryDBImpl:
  def __init__(self):
    # Structure: {key: {field: (value, expiration_time)}}
    self.data = {}
    
  def _is_expired(self, timestamp: int, key: str, field: str) -> bool:
      key_data = self.data.get(key, {})
      if field not in key_data:
        return True
      _, expiration_time = key_data[field]
      if expiration_time is not None and expiration_time < timestamp:
          key_data.pop(field)
          if not key_data:
              self.data.pop(key)
          return True
      return False

  # Internal set method (deletes the field if value is None and further deletes the key if there are no fields)
  def _set_int(self, timestamp: int, key: str, field: str, value: int | None, ttl: int | None = None) -> None:
    key_data = self.data.get(key, {})
    
    if value is None:
      key_data.pop(field, None)
      if len(key_data) == 0:
        self.data.pop(key, None)
        return
      self.data[key] = key_data

    key_data[field] = (value, timestamp + ttl if ttl else None)
    self.data[key] = key_data
  
  def get(self, timestamp: int, key: str, field: str) -> int | None:

    # Not explicitly mentioned, but makes sense
    if self._is_expired(timestamp, key, field):
      return None
    return self.data.get(key, {}).get(field, (None, None))[0]

  def scan(self, timestamp: int, key: str) -> list[str]:
    return self.scan_by_prefix(timestamp, key, '')

  def scan_by_prefix(self, timestamp: int, key: str, prefix: str) -> list[str]:
    key_data = self.data.get(key, {})
    # return list(map(lambda field: f'{field}({key_data[field]})',filter(lambda field: field.find(prefix) == 0, sorted(key_data.keys()))))
    return [
      f'{field}({key_data[field][0]})'
      for field in sorted(key_data.keys())
      if field.startswith(prefix) and not self._is_expired(timestamp, key, field)
    ]

  def set_with_ttl(self, timestamp: int, key: str, field: str, value: int, ttl: int) -> None:
    # assert type(value) == int
    return self._set_int(timestamp, key, field, value, ttl)

  def set(self, timestamp: int, key: str, field: str, value: int) -> None:
    # assert type(value) == int
    return self._set_int(timestamp, key, field, value)

  def compare_and_set(self, timestamp: int, key: str, field: str, expected_value: int, new_value: int) -> bool:
    if(self.get(timestamp, key, field) == expected_value):
      current_timestamp = self.data[key][field][1]
      self.set_with_ttl(timestamp, key, field, new_value, current_timestamp - timestamp if current_timestamp is not None else current_timestamp)
      return True
    return False

  def compare_and_set_with_ttl(self, timestamp: int, key: str, field: str, expected_value: int, new_value: int, ttl: int) -> bool:
    if(self.get(timestamp, key, field) == expected_value):
      self.set_with_ttl(timestamp, key, field, new_value, ttl)
      return True
    return False

  def compare_and_remove(self, timestamp: int, key: str, field: str, expected_value: int) -> bool:
    if(self.get(timestamp, key, field) == expected_value):
      self._set_int(timestamp, key, field, None)
      return True
    return False